In [1]:
from elasticsearch import Elasticsearch, helpers
import json
import numpy as np

In [2]:
es = Elasticsearch()

In [3]:
es.count(index='do', doc_type='do-comment')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 1186071}

In [4]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

from bs4 import BeautifulSoup

def strip_specific_tags(html, *tags):
    soup = BeautifulSoup(html)
    for tag in tags:
        for tag in soup.findAll(tag):
            tag.replaceWith("")

    return soup.get_text()

In [5]:
import string
from nltk.stem.snowball import SnowballStemmer
from sklearn import cross_validation

In [6]:
def prepareText(body):
    body = strip_tags(strip_specific_tags(body, 'code'))
    
    exclude = set(string.punctuation)
    body = ''.join(ch for ch in body if ch not in exclude)
    
    # split the text string, stem each word and append words to a list of words
    words = []
    stemmer = SnowballStemmer('english')
    for word in body.split():
        stemWord = stemmer.stem(word)
        words.append(stemWord)

    words = [x for x in words if not x.isdigit() and not x[0].isdigit()]
    return string.join(words)